In [5]:
!ollama pull llama2

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 8934d96d3f08... 100% ▕████████████████▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕████████████████▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕████████████████▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕████████████████▏   59 B                         
pulling fa304d675061... 100% ▕████████████████▏   91 B                         
pulling 42ba7f8a01dd... 100% ▕████████████████▏  557 B                         
verifyin

In [6]:

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [7]:
llm.invoke("turkish airline buggage policy?")

"\n Turkish Airlines baggage policy outlines the allowances and restrictions for luggage for both checked and carry-on bags. Here are some key points to consider:\n\nChecked Baggage:\n\n1. Weight: The maximum weight for each checked bag is 32 kg (70 lbs) for economy class and 45 kg (99 lbs) for business and first class.\n2. Size: The maximum dimensions for a checked bag are 158 cm (62 in) in length, width, and height.\n3. Number of bags: You can check up to 2 bags on Turkish Airlines, but this may vary depending on your fare type and travel dates.\n4. Fee: If you need to check more than two bags, you will be charged a fee for each additional bag.\n5. Special items: Turkish Airlines allows certain special items in checked luggage, such as sports equipment, musical instruments, and pets in carriers, but these may require additional documentation and fees.\n\nCarry-on Baggage:\n\n1. Size: The maximum size for a carry-on bag is 56 cm (22 in) in length, width, and height.\n2. Weight: There 

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [8]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a pro traveller."),
    ("user", "{input}")
])

In [9]:
#chain them togather

chain = prompt | llm 

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [10]:
chain.invoke({"input": "turkish airline buggage policy?"})

"\nAh, an astute traveler you are! *adjusts sunglasses* Turkish Airlines has a rather straightforward baggage policy, my friend. Here's the scoop:\n\n1. Checked Baggage: Turkish Airlines allows each passenger to check up to 2 pieces of baggage, with a maximum weight of 32 kg (70 lbs) per piece. The maximum dimensions for each piece of checked baggage are 62 inches (158 cm) in length + width + height.\n2. Cabin Baggage: You're allowed to bring one small bag or personal item on board, with a maximum weight of 8 kg (17.6 lbs). The maximum dimensions for cabin baggage are 45 inches (114 cm) in length + width + height.\n3. Excess Baggage: If you exceed the baggage allowance, you'll be charged a fee. The rates vary depending on the route and your frequent flyer status. Check their website for more details.\n4. Special Baggage: Turkish Airlines has specific rules for transporting certain items, such as surfboards, skis, and musical instruments. Check their website for more information on how 

In [11]:
#The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()



In [12]:
#We can now add this to the previous chain:

chain = prompt | llm | output_parser

In [13]:
chain.invoke({"input": "turkish airline buggage policy?"})

'\nAh, an inquiry about Turkish Airlines\' baggage policy, you say? Well, my friend, let me tell you, they have quite the interesting and flexible policy when it comes to checking and carrying luggage. 😉\n\nTurkish Airlines allows each passenger to check in a maximum of 2 bags, with a combined weight limit of 30 kg (66 lbs) per bag. However, if you\'re traveling in Economy Class, you\'re allowed to bring an additional small bag weighing up to 8 kg (17.6 lbs) for a total of 38 kg (84 lbs) of luggage.\n\nBut wait, there\'s more! If you\'re traveling in Premium Class or Business Class, you can bring an unlimited number of bags, each weighing up to 30 kg (66 lbs). Now, that\'s what I call luxury travel! 😎\n\nAs for carry-on luggage, Turkish Airlines allows you to bring one small bag with a maximum weight limit of 8 kg (17.6 lbs) and a size limit of 50 cm x 40 cm x 20 cm (20 in x 16 in x 8 in). Anything larger than that will have to be checked in.\n\nNow, I know what you\'re thinking: "What

# Retrieval Chain

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [14]:
!pip install beautifulsoup4

To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see this documentation.



In [15]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.turkishairlines.com/en-int/help-center/")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

In [16]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [17]:
#Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.

#First we need to install the required packages for that:

!pip install faiss-cpu

In [18]:
#Then we can build our index:

from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:



In [20]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "turkish airline buggage policy?",
    "context": [Document(page_content="turkish airline buggage policy")]
})

"\nAnswer: Turkish Airlines allows passengers to check-in baggage with a maximum weight of 32 kg (70 lbs) per person, and a maximum size of 158 cm (62 in) in length + width + height. Additionally, each passenger is allowed to bring one small bag or laptop bag on board for free, as long as it fits under the seat in front of them. Any additional bags will be subject to a fee. It's best to check the airline's website or consult with their customer service for the most up-to-date and detailed information on their baggage policies."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.



In [21]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key


In [22]:

response = retrieval_chain.invoke({"input": "turkish airline buggage policy"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Turkish Airlines baggage policy outlines the allowable dimensions and weights for carry-on and checked bags, as well as any additional fees or restrictions. Here are some key points to consider when planning your trip with Turkish Airlines:

Carry-On Baggage:

* Maximum size: 56 cm x 45 cm x 25 cm (22 in x 17.7 in x 9.8 in)
* Maximum weight: 10 kg (22 lbs)
* Number of carry-on bags: Unlimited

Checked Baggage:

* Maximum size: 158 cm x 23 cm x 70 cm (62 in x 9 in x 27.5 in) for luggage and 292 cm x 48 cm x 110 cm (115 in x 19 in x 43 in) for surfboards, bicycles, and other sports equipment
* Maximum weight: 23 kg (50 lbs) for luggage and 32 kg (70 lbs) for surfboards, bicycles, and other sports equipment
* Number of checked bags: Unlimited

Additional Baggage Fees:

* For carry-on bags that exceed the size limit: $50-$100 USD
* For checked bags that exceed the weight limit: $200-$300 USD
* For oversized or overweight baggage: $100-$200 USD per piece

Special Baggage Items:

* Surfboard

# Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

1. The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.
2. he final LLM chain should likewise take the whole history into account

## Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.



In [23]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [24]:
#We can test this out by passing in an instance where the user asks a follow-up question.

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="is onlye 32 kg is allowed on turkish airline?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more"
})

[Document(page_content='We are delighted to assist our passengers. You can find detailed information on all our services, including baggage allowance, online services and feedback, as well as passengers’ most frequently asked questions, at the Turkish Airlines Help Center. You can easily find the answers to all your questions here.    Frequently asked questions Baggage Miles&Smiles Online services Additional services Discover Contact us       Frequently asked questions    You can find detailed information regarding the processes before, during and after the flight from the Turkish Airlines Help Center.     Check-in information > Travelling with pets > Infants and children > Transfer and transit passengers > Payment > Reservations and bookings > Fare rules > Book with us >   See all       Baggage    You can carry out baggage-related transactions through the Turkish Airlines Help Center. The Help Center also provides information on baggage restrictions.     Baggage tracking > Baggage dis

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [26]:
#We can now test this out end-to-end:

chat_history = [HumanMessage(content="is onlye 32 kg is allowed on turkish airline?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more"
})

{'chat_history': [HumanMessage(content='is onlye 32 kg is allowed on turkish airline?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me more',
 'context': [Document(page_content='Enable Javascript ×         Skip to main content        Toggle navigation             BOOK&MANAGE      BOOK     MANAGE     ADDITIONAL SERVICES        Book a flight   Mobile app   Hold the price   TK Wallet       Manage booking   Check-in   Flight status       Seat selection   Extra baggage   Traveling with pets   Business Upgrade   Sports equipment   Rent a car   Book a hotel   Travel insurance   UAE E-Visa   See all          EXPERIENCE      CABIN CLASSES     EXPERIENCE     MORE        Business Class   Economy Class       Dining on-board   Inflight entertainment   Fleet       Turkish Airlines Lounge   Stopover Istanbul   Touristanbul   Exclusive Drive   PressReader   Istanbul Airport   See all          DEALS&DESTINATIONS      OFFERS     DESTINATIONS     DISCOVER         Best flight deals    Special offers   

We can see that this gives a coherent answer - we've successfully turned our retrieval chain into a chatbot!



# Agent

We've so far created examples of chains - where each step is known ahead of time. The final thing we will create is an agent - where the LLM decides what steps to take.

NOTE: for this example we will only show how to create an agent using OpenAI models, as local models are not reliable enough yet.

One of the first things to do when building an agent is to decide what tools it should have access to. For this example, we will give the agent access to two tools:

1. The retriever we just created. This will let it easily answer questions about LangSmith
2. A search tool. This will let it easily answer questions that require up-to-date information.


First, let's set up a tool for the retriever we just created: